In [7]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime


C:\Users\benna435\AppData\Roaming\Python\Python312\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\benna435\AppData\Roaming\Python\Python312\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\benna435\AppData\Roaming\Python\Python312\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto

In [9]:
import yfinance as yf
import pandas as pd
import time

# ============================================
# 1. ORIGINAL TICKERS (YOUR DOWNLOADED LIST)
# ============================================

tickers = [
"MMM","AOS","ABT","ABBV","ACN","ADBE","AMD","AES","AFL","A","AAP","AMT","AWK",
"AMP","ABC","AME","AMGN","APH","ADI","ANSS","ANTM","AON","APA","AAPL","AMAT",
"APTV","ADM","ANET","AJG","AIZ","T","ATO","ADSK","AZO","AVB","AVY","BKR","BLL",
"BAC","BK","BAX","BDX","BRO","BBWI","BR","BMY","AVGO","BRK-B","CDNS","CZR",
"CPB","COF","CAH","KMX","CCL","CARR","CTLT","CAT","CBOE","CBRE","CDW","CE","CNC",
"CNP","CDAY","CF","CRL","SCHW","CHTR","CVX","CMG","CB","CHD","CI","CINF","CTAS",
"CSCO","C","CFG","CTXS","CLX","CME","CMS","KO","CTSH","CL","CMCSA","CMA","CAG",
"COP","ED","STZ","CEG","COO","CPRT","GLW","CTVA","CSX","CMI","CVS","DHI","DHR",
"DRI","DVA","DE","DAL","XRAY","DVN","DXCM","FANG","DLR","DFS","DIS","DG","DLTR",
"D","DPZ","DOV","DOW","DTE","DUK","DRE","DD","DXC","EMN","ETN","EBAY","ECL",
"EIX","EW","EA","EMR","ENPH","ETR","EOG","EFX","EQIX","EQR","ESS","EL","ETSY",
"RE","EXC","EXPE","EXPD","EXR","XOM","FFIV","FDS","FIS","FITB","FE","FRC","FISV",
"FLT","FMC","F","FTNT","FTV","FBHS","FOXA","FOX","BEN","FCX","GRMN","IT","GNRC",
"GD","GE","GIS","GM","GPC","GILD","GPN","GL","GS","HAL","HIG","HAS","HCA","PEAK",
"HSIC","HSY","HES","HPE","HLT","HOLX","HD","HON","HRL","HST","HWM","HPQ","HUM",
"HBAN","HII","IBM","IEX","IDXX","ITW","ILMN","INCY","IR","INTC","ICE","IFF","IP",
"IPG","INTU","ISRG","IVZ","IPGP","IQV","IRM","JBHT","J","SJM","JNJ","JCI","JPM",
"JNPR","K","KEY","KEYS","KMB","KIM","KMI","KLAC","KHC","KR","LHX","LH","LRCX",
"LW","LVS","LDOS","LEN","LLY","LNC","LIN","LYV","LKQ","LMT","L","LOW","LUMN",
"LYB","MTB","MRO","MPC","MKTX","MAR","MMC","MLM","MAS","MA","MTCH","MKC","MCD",
"MCK","MDT","MRK","META","MTD","MGM","MCHP","MU","MSFT","MAA","MRNA","MHK","TAP",
"MDLZ","MPWR","MNST","MCO","MS","MOS","MSI","MSCI","NDAQ","NTAP","NFLX","NWL",
"NEM","NWSA","NWS","NEE","NLSN","NKE","NI","NSC","NTRS","NOC","NLOK","NVR","NXPI",
"ORLY","OXY","ODFL","OMC","OKE","ORCL","OGN","OTIS","PCAR","PKG","PH","PAYX",
"PAYC","PFE","PM","PSX","PNW","PXD","PNC","POOL","PPG","PPL","PFG","PG","PGR",
"PLD","PRU","PEG","PSA","PHM","PVH","QRVO","QCOM","DGX","RL","ROP","ROST","RCL",
"SPGI","CRM","SBAC","SLB","STX","SEE","SRE","SHW","SPG","SWKS","SNA","SO","LUV",
"SWK","SBUX","STT","STE","SYK","SIVB","SYF","SNPS","SYY","TMUS","TROW","TT","TDY",
"TFX","TER","TSLA","TXN","TXT","TMO","TJX","TSCO","TTWO","TDG","TRV","TRMB",
"TFC","TWTR","TYL","TSN","UDR","ULTA","USB","UAA","UA","UNP","UAL","UPS","URI",
"UNH","UHS","VFC","VTR","VTRS","VICI","VZ","V","VMW","VNO","VMC","WRB","WAB",
"WMT","WBA","DIS","WM","WAT","WEC","WFC","WELL","WDC","WHR","WMB","WLTW","WYNN",
"XEL","XLNX","XYL","YUM","ZBRA","ZBH","ZION","ZTS"
]


# ============================================
# 2. FIX MAP (RENAME BROKEN TICKERS)
# ============================================

ticker_fix = {
    "ABC": "COR",     # AmerisourceBergen → Cencora
    "ANTM": "ELV",    # Anthem → Elevance Health
    "BLL": "BALL",    # Ball Corp
    "CDAY": "DAY",    # Ceridian → Dayforce
    "FLT": "CORP",    # renamed to CORP
    "FBHS": "FBIN",   # Fortune Brands
    "PEAK": "DOC",    # merged → Physicians Realty
    "RE": "EG",       # renamed
    "NLOK": "GEN",    # Norton → Gen Digital
    "WLTW": "WTW",    # Willis Towers Watson → WTW
}

# ============================================
# 3. REMOVE LIST (DELISTED / MERGED / DEAD)
# ============================================

remove_list = [
    "ANSS","CTLT","CTXS","DFS","DRE","FRC","HES","JNPR","MRO",
    "NLSN","PXD","SIVB","TWTR","VMW","WBA","XLNX"
]


# ============================================
# 4. CLEAN FINAL TICKER LIST
# ============================================

cleaned_tickers = []
for t in tickers:
    if t in remove_list:
        continue
    cleaned_tickers.append(ticker_fix.get(t, t))

len(cleaned_tickers)


404

In [10]:
all_data = {}
start_date = "2000-01-01"
failed = []

for t in cleaned_tickers:
    try:
        print("Downloading:", t)
        df = yf.download(t, start=start_date, auto_adjust=True, progress=False)

        # require enough history
        if df.shape[0] < 4000:
            raise Exception("Insufficient history")

        all_data[t] = df["Close"]
    except Exception as e:
        print("FAILED:", t, e)
        failed.append(t)
    
    time.sleep(0.10)  # avoid rate limit

print("\nSuccess:", len(all_data), "stocks")
print("Failed:", len(failed), failed)


Downloading: MMM
Downloading: AOS
Downloading: ABT
Downloading: ABBV
FAILED: ABBV Insufficient history
Downloading: ACN
Downloading: ADBE
Downloading: AMD
Downloading: AES
Downloading: AFL
Downloading: A
Downloading: AAP
Downloading: AMT
Downloading: AWK
Downloading: AMP
Downloading: COR
Downloading: AME
Downloading: AMGN
Downloading: APH
Downloading: ADI
Downloading: ELV
Downloading: AON
Downloading: APA
Downloading: AAPL
Downloading: AMAT
Downloading: APTV
FAILED: APTV Insufficient history
Downloading: ADM
Downloading: ANET
FAILED: ANET Insufficient history
Downloading: AJG
Downloading: AIZ
Downloading: T
Downloading: ATO
Downloading: ADSK
Downloading: AZO
Downloading: AVB
Downloading: AVY
Downloading: BKR
Downloading: BALL
Downloading: BAC
Downloading: BK
Downloading: BAX
Downloading: BDX
Downloading: BRO
Downloading: BBWI
Downloading: BR
Downloading: BMY
Downloading: AVGO
Downloading: BRK-B
Downloading: CDNS
Downloading: CZR
FAILED: CZR Insufficient history
Downloading: CPB
Downloa

In [25]:
all_data

{'MMM': Ticker             MMM
 Date                  
 2000-01-03   19.343809
 2000-01-04   18.575178
 2000-01-05   19.113220
 2000-01-06   20.650475
 2000-01-07   21.060404
 ...                ...
 2025-11-11  167.948334
 2025-11-12  170.348022
 2025-11-13  169.889984
 2025-11-14  167.580002
 2025-11-17  165.389999
 
 [6509 rows x 1 columns],
 'AOS': Ticker            AOS
 Date                 
 2000-01-03   2.265212
 2000-01-04   2.232574
 2000-01-05   2.226045
 2000-01-06   2.193405
 2000-01-07   2.284798
 ...               ...
 2025-11-11  65.790001
 2025-11-12  66.370003
 2025-11-13  66.089996
 2025-11-14  64.739998
 2025-11-17  63.310001
 
 [6509 rows x 1 columns],
 'ABT': Ticker             ABT
 Date                  
 2000-01-03    8.134056
 2000-01-04    7.901651
 2000-01-05    7.887127
 2000-01-06    8.163101
 2000-01-07    8.250257
 ...                ...
 2025-11-11  127.000000
 2025-11-12  128.820007
 2025-11-13  129.259995
 2025-11-14  130.589996
 2025-11-17  130.029999
